<div align="center">

# **DELIVERY 2**
## **Indexing and Evaluation**

</div>

---

## **PART 1: Indexing**

### **STEP 1 — Build inverted index:**

### **Main Code**

In [1]:
import json
from pathlib import Path
from collections import defaultdict
from typing import Dict, List, Set, Any, Iterable
from pathlib import Path
import sys

NOTEBOOK_DIR = Path().resolve()
REPO_ROOT = NOTEBOOK_DIR.parents[1] if NOTEBOOK_DIR.name in {"part_1", "part_2"} else NOTEBOOK_DIR
sys.path.append(str(REPO_ROOT / "project_progress"))
from utils.preprocessing import preprocess_text_field



# Path
NOTEBOOK_DIR = Path().resolve()
REPO_ROOT = NOTEBOOK_DIR.parents[1]          
DATA_DIR = REPO_ROOT / "data"
INPUT = DATA_DIR / "fashion_products_dataset_enriched.json"

INDEX_DIR = DATA_DIR / "index"
INDEX_DIR.mkdir(parents=True, exist_ok=True)

INDEX_FILE = INDEX_DIR / "boolean_inverted_index.json"
DOCMAP_FILE = INDEX_DIR / "docid_pid_map.json"
FIELDS_FILE = INDEX_DIR / "indexed_fields.json"

print(f"Reading enriched dataset: {INPUT}")
print(f"Index will be saved in:   {INDEX_DIR}")


if not INPUT.exists():
    raise FileNotFoundError(f"Enriched dataset not found: {INPUT}")
docs: List[Dict[str, Any]] = json.loads(INPUT.read_text(encoding="utf-8"))
print(f"Loaded {len(docs)} docs")


INDEXED_TEXT_FIELDS = [
    "title_clean",
    "description_clean",
    "metadata_clean",   
]


# doc_id is an integer, stable order = index in list
docid_to_pid: Dict[int, str] = {}
pid_to_docid: Dict[str, int] = {}

for i, r in enumerate(docs):
    pid = r.get("pid")
    if not pid:
        pid = r.get("_id", f"missing_pid_{i}")
    docid_to_pid[i] = pid
    pid_to_docid[pid] = i

def _doc_tokens(record: Dict[str, Any], fields: Iterable[str]) -> List[str]:
    toks: List[str] = []
    for f in fields:
        val = record.get(f)
        if not val:
            continue
        # We already have cleaned strings; just split.
        toks.extend(str(val).split())
    return toks


# Build inverted index 
vocab: Dict[str, Set[int]] = defaultdict(set)

for doc_id, rec in enumerate(docs):
    tokens = _doc_tokens(rec, INDEXED_TEXT_FIELDS)
    if not tokens:
        continue
    # Use unique terms per doc for Boolean presence posting
    for term in set(tokens):
        vocab[term].add(doc_id)

# Convert sets to sorted lists for compactness and efficient AND intersections
inverted_index: Dict[str, List[int]] = {t: sorted(list(s)) for t, s in vocab.items()}
print(f"Vocabulary size: {len(inverted_index):,}")


INDEX_FILE.write_text(json.dumps(inverted_index), encoding="utf-8")
DOCMAP_FILE.write_text(json.dumps({"docid_to_pid": docid_to_pid}, ensure_ascii=False), encoding="utf-8")
FIELDS_FILE.write_text(json.dumps({"indexed_fields": INDEXED_TEXT_FIELDS}, ensure_ascii=False, indent=2), encoding="utf-8")

print(f"Saved inverted index to: {INDEX_FILE}")
print(f"Saved doc map         to: {DOCMAP_FILE}")
print(f"Saved fields          to: {FIELDS_FILE}")


REQUIRED_OUTPUT_FIELDS = [
    "pid", "title", "description", "brand", "category", "sub_category",
    "product_details", "seller", "out_of_stock", "selling_price", "discount",
    "actual_price", "average_rating", "url"
]

def _query_tokens(q: str) -> List[str]:
    # Use the same normalization and stemming pipeline as Step 1
    proc = preprocess_text_field(q or "")
    return proc["tokens"]

def _intersect_sorted(a: List[int], b: List[int]) -> List[int]:
    """Intersect two sorted posting lists."""
    i=j=0
    out: List[int] = []
    while i < len(a) and j < len(b):
        if a[i] == b[j]:
            out.append(a[i])
            i+=1; j+=1
        elif a[i] < b[j]:
            i+=1
        else:
            j+=1
    return out

def search_and(query: str, fields: List[str] = None, k: int = 20) -> List[Dict[str, Any]]:
    """
    Conjunctive (AND) Boolean search.
    Every returned doc must contain ALL query terms (after preprocessing).
    Returns up to k full records with the required fields (when present).
    """
    _ = fields  # kept for future extension; current index already built over INDEXED_TEXT_FIELDS
    q_terms = _query_tokens(query)
    if not q_terms:
        return []

    # Load postings lists; if any term not in vocab -> empty result
    postings_lists: List[List[int]] = []
    for t in q_terms:
        p = inverted_index.get(t)
        if not p:
            return []
        postings_lists.append(p)

    # Intersect from shortest to longest for speed
    postings_lists.sort(key=len)
    result_ids = postings_lists[0]
    for pl in postings_lists[1:]:
        result_ids = _intersect_sorted(result_ids, pl)
        if not result_ids:
            break

    # Map to records and keep only required output fields (when present)
    out: List[Dict[str, Any]] = []
    for did in result_ids[:k]:
        rec = docs[did]
        # Build a thin view with required fields (include only those present)
        view = {f: rec.get(f) for f in REQUIRED_OUTPUT_FIELDS if f in rec}
        # Always include pid
        if "pid" not in view:
            view["pid"] = rec.get("pid") or docid_to_pid.get(did)
        out.append(view)
    return out

Reading enriched dataset: C:\Users\Pol\Documents\POL\UNI\WEB\irwa-search-engine\data\fashion_products_dataset_enriched.json
Index will be saved in:   C:\Users\Pol\Documents\POL\UNI\WEB\irwa-search-engine\data\index
Loaded 28080 docs
Vocabulary size: 9,048
Saved inverted index to: C:\Users\Pol\Documents\POL\UNI\WEB\irwa-search-engine\data\index\boolean_inverted_index.json
Saved doc map         to: C:\Users\Pol\Documents\POL\UNI\WEB\irwa-search-engine\data\index\docid_pid_map.json
Saved fields          to: C:\Users\Pol\Documents\POL\UNI\WEB\irwa-search-engine\data\index\indexed_fields.json


### **STEP 2 — Propose test queries:**

In [2]:
import random

def df(term: str) -> int:
    """Document frequency of a term (0 if absent)."""
    return len(inverted_index.get(term, []))

def in_vocab(token: str) -> bool:
    return token in inverted_index

def stem_phrase(phrase: str) -> list[str]:
    return preprocess_text_field(phrase)["tokens"]

def phrase_ok(phrase: str) -> bool:
    """All tokens exist in vocab AND the AND-query returns at least one result."""
    toks = stem_phrase(phrase)
    if not toks or not all(in_vocab(t) for t in toks):
        return False
    return len(search_and(phrase, k=1)) > 0

def term_popularity_score(tokens: list[str]) -> int:
    """Sum of dfs for quick 'popularity' proxy."""
    return sum(df(t) for t in tokens)

# Candidate lexicons (raw phrases)
gender_phrases  = ["men", "women"]
type_phrases    = [
    "jeans", "shirt", "t shirt", "hoodie", "sweatshirt",
    "track pants", "kurta", "dress", "jacket"
]
color_phrases   = ["black", "blue", "white", "grey", "red", "green", "pink"]
material_phrases= ["cotton", "polyester", "denim", "linen", "silk"]
fit_phrases     = ["slim", "skinny", "regular", "straight", "high waist"]
style_phrases   = ["printed", "solid", "striped", "floral"]

# Keep only phrases whose tokens exist in vocab (post-stemming)
def filter_vocab(phrases):
    good = []
    for p in phrases:
        toks = stem_phrase(p)
        if toks and all(in_vocab(t) for t in toks):
            good.append((p, toks, term_popularity_score(toks)))
    # Sort by popularity descending (PRP)
    return sorted(good, key=lambda x: x[2], reverse=True)

gender_ok   = filter_vocab(gender_phrases)
types_ok    = filter_vocab(type_phrases)
colors_ok   = filter_vocab(color_phrases)
materials_ok= filter_vocab(material_phrases)
fits_ok     = filter_vocab(fit_phrases)
styles_ok   = filter_vocab(style_phrases)

# Compose candidate query templates: (gender) + (type) + (attribute sets)
templates = [
    ["{g}", "{m}", "{t}", "{c}"],                 # gender + material + type + color
    ["{g}", "{t}", "{f}", "{c}"],                 # gender + type + fit + color
    ["{g}", "full sleeve", "{t}", "{m}"],         # gender + sleeve attr + type + material
    ["{g}", "{t}", "{s}", "{c}"],                 # gender + type + style + color
    ["{g}", "high waist", "{t}", "{c}"],          # gender + high waist + type + color
]

def pick(pop_list, k=1):
    return [p[0] for p in pop_list[:k]] if pop_list else []

# Generate diverse, popular queries that actually return hits
proposed = []
attempts = 0
seen_main_types = set()

while len(proposed) < 5 and attempts < 200:
    attempts += 1
    tpl = random.choice(templates)

    g = pick(gender_ok, 1) or ["women"]
    t = pick(types_ok, 1) or ["jeans"]
    m = pick(materials_ok, 1) or ["cotton"]
    c = pick(colors_ok, 1) or ["blue"]
    f = pick(fits_ok, 1) or ["slim"]
    s = pick(styles_ok, 1) or ["printed"]

    phrase = " ".join(
        x.format(g=g[0], t=t[0], m=m[0], c=c[0], f=f[0], s=s[0])
        for x in tpl
    )
    phrase = " ".join(phrase.split())  # clean double spaces

    # Require the AND query to return hits and encourage diversity by not repeating the same main type too much.
    main_type = t[0]
    if phrase_ok(phrase):
        if sum(1 for q in proposed if main_type in q) < 2:
            proposed.append(phrase)

# Fallbacks (just in case)
fallbacks = [
    "women cotton kurta straight",
    "men slim fit formal shirt",
    "women high waist blue jeans",
    "men running shoes black",
    "women printed dress floral"
]
for fb in fallbacks:
    if len(proposed) >= 5:
        break
    if fb not in proposed and phrase_ok(fb):
        proposed.append(fb)

# Deduplicate and trim to 5
seen = set()
unique_proposed = []
for q in proposed:
    if q not in seen:
        unique_proposed.append(q)
        seen.add(q)
proposed = unique_proposed[:5]

print("=== Proposed Test Queries (data-driven) ===")
for i, q in enumerate(proposed, 1):
    hits = len(search_and(q, k=50))
    toks = stem_phrase(q)
    score = term_popularity_score(toks)
    print(f"{i}. {q}  | tokens={toks} | DF-score={score} | matches≈{hits}")

# Store for later evaluation/report
TEST_QUERIES_FILE = DATA_DIR / "index" / "proposed_test_queries.json"
TEST_QUERIES_FILE.write_text(json.dumps({"queries": proposed}, indent=2), encoding="utf-8")
print(f"\nSaved queries to: {TEST_QUERIES_FILE}")

=== Proposed Test Queries (data-driven) ===
1. men shirt regular blue  | tokens=['men', 'shirt', 'regular', 'blue'] | DF-score=53690 | matches≈50
2. men shirt solid blue  | tokens=['men', 'shirt', 'solid', 'blue'] | DF-score=50331 | matches≈50
3. women cotton kurta straight  | tokens=['women', 'cotton', 'kurta', 'straight'] | DF-score=35775 | matches≈50
4. men slim fit formal shirt  | tokens=['men', 'slim', 'fit', 'formal', 'shirt'] | DF-score=57466 | matches≈50
5. men running shoes black  | tokens=['men', 'run', 'shoe', 'black'] | DF-score=21967 | matches≈28

Saved queries to: C:\Users\Pol\Documents\POL\UNI\WEB\irwa-search-engine\data\index\proposed_test_queries.json


### **STEP 3 — Ranking our results:**

### **Main Code**

In [3]:
import math
from typing import Tuple

# Configuration
INDEXED_TEXT_FIELDS = ["title_clean", "description_clean", "metadata_clean"]
FIELD_WEIGHTS: Dict[str, float] = {
    "title_clean": 2.0,     
    "description_clean": 1.0,  
    "metadata_clean": 0.7,     
}

# Utilities 
def _tokens_from_fields(record: Dict[str, Any], fields: Iterable[str]) -> Dict[str, Dict[str, int]]:
    """
    Return per-field raw term frequencies:
        { field_name : { term : count_in_that_field } }
    """
    per_field_counts: Dict[str, Dict[str, int]] = {}
    for f in fields:
        txt = record.get(f)
        if not txt:
            continue
        counts: Dict[str, int] = defaultdict(int)
        for t in str(txt).split():
            counts[t] += 1
        if counts:
            per_field_counts[f] = dict(counts)
    return per_field_counts

def _tf_log2(freq: float) -> float:
    """1 + log2(freq) if freq>0 else 0."""
    if freq <= 0:
        return 0.0
    return 1.0 + math.log(freq, 2)

def _idf_log2(df_i: int, N: int) -> float:
    """idf = log2(N / df_i); assumes df_i >= 1."""
    if df_i <= 0 or N <= 0:
        return 0.0
    return math.log(N / df_i, 2)

# Build TF postings & df
N = len(docs)

# term -> { doc_id : (field-weighted raw frequency) }
tf_postings: Dict[str, Dict[int, float]] = defaultdict(dict)
# term -> document frequency
df_counts: Dict[str, int] = defaultdict(int)

for doc_id, rec in enumerate(docs):
    per_field = _tokens_from_fields(rec, INDEXED_TEXT_FIELDS)

    term_freq_weighted: Dict[str, float] = defaultdict(float)
    for field_name, counts in per_field.items():
        w_f = FIELD_WEIGHTS.get(field_name, 1.0)
        for term, f_ij_f in counts.items():
            term_freq_weighted[term] += w_f * f_ij_f

    for term, f_ij in term_freq_weighted.items():
        tf_postings[term][doc_id] = f_ij

# df_i = number of docs where term appears
for term, posting in tf_postings.items():
    df_counts[term] = len(posting)

# Precompute document norms
doc_norms: List[float] = [0.0] * N
for term, posting in tf_postings.items():
    idf_i = _idf_log2(df_counts[term], N)
    if idf_i == 0.0:
        continue
    for d_id, f_ij in posting.items():
        w_dt = _tf_log2(f_ij) * idf_i
        if w_dt != 0.0:
            doc_norms[d_id] += w_dt * w_dt

doc_norms = [math.sqrt(v) if v > 0 else 0.0 for v in doc_norms]

# Ranked search
def search_tfidf(query: str, k: int = 20) -> List[Dict[str, Any]]:
    """
    Rank documents by cosine similarity with TF-IDF (base-2 logs, no smoothing).
    Returns top-k with 'score' plus the required output fields.
    """
    q_proc = preprocess_text_field(query or "")
    q_terms = q_proc["tokens"]
    if not q_terms:
        return []

    # query term frequencies
    q_tf: Dict[str, int] = defaultdict(int)
    for t in q_terms:
        q_tf[t] += 1

    # build query vector
    q_weights: Dict[str, float] = {}
    q_norm_sq = 0.0
    for t, f_q in q_tf.items():
        df_i = df_counts.get(t, 0)
        if df_i <= 0:
            continue  # unseen term
        w_t = _tf_log2(f_q) * _idf_log2(df_i, N)
        if w_t == 0.0:
            continue
        q_weights[t] = w_t
        q_norm_sq += w_t * w_t

    q_norm = math.sqrt(q_norm_sq) if q_norm_sq > 0 else 0.0
    if q_norm == 0.0:
        return []

    # sparse dot product over postings of query terms
    scores: Dict[int, float] = defaultdict(float)
    for t, w_t in q_weights.items():
        posting = tf_postings.get(t)
        if not posting:
            continue
        idf_i = _idf_log2(df_counts[t], N)
        if idf_i == 0.0:
            continue
        for d_id, f_ij in posting.items():
            w_dt = _tf_log2(f_ij) * idf_i
            if w_dt != 0.0:
                scores[d_id] += w_t * w_dt

    # cosine normalization and rank
    ranked: List[Tuple[int, float]] = []
    for d_id, dot in scores.items():
        denom = doc_norms[d_id] * q_norm
        if denom > 0:
            ranked.append((d_id, dot / denom))
    ranked.sort(key=lambda x: x[1], reverse=True)

    results: List[Dict[str, Any]] = []
    for d_id, sc in ranked[:k]:
        rec = docs[d_id]
        view = {f: rec.get(f) for f in REQUIRED_OUTPUT_FIELDS if f in rec}
        if "pid" not in view:
            view["pid"] = rec.get("pid")
        view["score"] = float(sc)
        results.append(view)
    return results

# AND-filtered TF-IDF
def search_tfidf_and(query: str, k: int = 20) -> List[Dict[str, Any]]:
    """
    Conjunctive AND filter first (Boolean), then TF-IDF rank within the survivors.
    Helpful if your teacher wants AND semantics even for ranking.
    """
    # Candidate set via Boolean AND
    cand = search_and(query, k=10_000)
    if not cand:
        return []

    cand_pids = {r["pid"] for r in cand if r.get("pid")}
    cand_ids = {i for i, r in enumerate(docs) if r.get("pid") in cand_pids}

    # Build query vector (same as search_tfidf)
    q_proc = preprocess_text_field(query or "")
    q_terms = q_proc["tokens"]
    if not q_terms:
        return []

    q_tf: Dict[str, int] = defaultdict(int)
    for t in q_terms:
        q_tf[t] += 1

    q_weights: Dict[str, float] = {}
    q_norm_sq = 0.0
    for t, f_q in q_tf.items():
        df_i = df_counts.get(t, 0)
        if df_i <= 0:
            continue
        w_t = _tf_log2(f_q) * _idf_log2(df_i, N)
        if w_t == 0.0:
            continue
        q_weights[t] = w_t
        q_norm_sq += w_t * w_t

    q_norm = math.sqrt(q_norm_sq) if q_norm_sq > 0 else 0.0
    if q_norm == 0.0:
        return []

    scores: Dict[int, float] = defaultdict(float)
    for t, w_t in q_weights.items():
        posting = tf_postings.get(t)
        if not posting:
            continue
        idf_i = _idf_log2(df_counts[t], N)
        if idf_i == 0.0:
            continue
        for d_id, f_ij in posting.items():
            if d_id not in cand_ids:
                continue
            w_dt = _tf_log2(f_ij) * idf_i
            if w_dt != 0.0:
                scores[d_id] += w_t * w_dt

    ranked: List[Tuple[int, float]] = []
    for d_id, dot in scores.items():
        denom = doc_norms[d_id] * q_norm
        if denom > 0:
            ranked.append((d_id, dot / denom))
    ranked.sort(key=lambda x: x[1], reverse=True)

    results: List[Dict[str, Any]] = []
    for d_id, sc in ranked[:k]:
        rec = docs[d_id]
        view = {f: rec.get(f) for f in REQUIRED_OUTPUT_FIELDS if f in rec}
        if "pid" not in view:
            view["pid"] = rec.get("pid")
        view["score"] = float(sc)
        results.append(view)
    return results

# Persist ranked results
def save_ranked_results(out_path: Path, queries: Dict[str, List[str]], use_and_filter: bool = False, k: int = 20) -> Path:
    """
    Save ranked results for groups of queries.
    queries = {"provided": [q1, q2, ...], "proposed": [q3, ...]}
    """
    out = {"provided_queries": {}, "proposed_queries": {}}
    ranker = search_tfidf_and if use_and_filter else search_tfidf

    for group, qlist in queries.items():
        for q in qlist:
            out_key = "provided_queries" if group == "provided" else "proposed_queries"
            out[out_key][q] = ranker(q, k=k)

    out_path.write_text(json.dumps(out, ensure_ascii=False, indent=2), encoding="utf-8")
    return out_path

### **Testing**

In [4]:
# Quick demo on course queries
for q in ["women full sleeve sweatshirt cotton", "men slim jeans blue"]:
    top = search_tfidf(q, k=3)
    print(f"\nTF-IDF (log2) top for: {q!r}")
    for r in top:
        print(f"  {r['score']:.4f} | {r.get('pid')} | {(r.get('title') or '')[:80]}")

# Save ranked results for report/repro
RANKED_OUT = (DATA_DIR / "index" / "ranked_results.json")
queries_for_report = {
    "provided": [
        "women full sleeve sweatshirt cotton",
        "men slim jeans blue",
    ],
    # Optionally load your proposed queries file if it exists
}
pq_file = DATA_DIR / "index" / "proposed_test_queries.json"
if pq_file.exists():
    try:
        queries_for_report["proposed"] = json.loads(pq_file.read_text(encoding="utf-8"))["queries"]
    except Exception:
        pass

out_path = save_ranked_results(RANKED_OUT, queries_for_report, use_and_filter=False, k=20)
print(f"\nRanked results saved to: {out_path}")


TF-IDF (log2) top for: 'women full sleeve sweatshirt cotton'
  0.9074 | SWSFZVTTQCB4SJ7F | Full Sleeve Solid Women Sweatshirt
  0.8760 | SWSFQGS456JAZCQB | Full Sleeve Printed Women Sweatshirt
  0.8724 | SWSFYTYMNTBNARUN | Full Sleeve Solid Women Sweatshirt

TF-IDF (log2) top for: 'men slim jeans blue'
  0.7168 | JEAFSKYHZHSZZC9S | Slim Men Blue Jeans
  0.7147 | JEAFRAQXEKGUPNUN | Slim Men Blue Jeans
  0.7096 | JEAFQF6JBUSEXHVF | Slim Men Blue Jeans

Ranked results saved to: C:\Users\Pol\Documents\POL\UNI\WEB\irwa-search-engine\data\index\ranked_results.json


## **PART 2: Evaluation**

### **STEP 1 — Implementing Metrics:**

In [5]:
from typing import List, Dict, Any, Tuple

def precision_at_k(rel_ranked: List[int], k: int) -> float:
    """P@K = (# relevant in top-K) / K."""
    if k <= 0:
        return 0.0
    top = rel_ranked[:k]
    return sum(top) / k

def recall_at_k(rel_ranked: List[int], k: int, total_relevant: int) -> float:
    """R@K = (# relevant in top-K) / (# relevant in the corpus for this query)."""
    if total_relevant <= 0:
        return 0.0
    return sum(rel_ranked[:k]) / total_relevant

def average_precision_at_k(rel_ranked: List[int], k: int, total_relevant: int) -> float:
    """
    AP@K = Average of precisions at ranks where a relevant item occurs, up to K,
           divided by the TOTAL number of relevant documents for the query.
    If there are no relevant docs, AP is 0 by convention.
    """
    if total_relevant <= 0:
        return 0.0
    ap_sum = 0.0
    for i in range(1, min(k, len(rel_ranked)) + 1):
        if rel_ranked[i - 1] == 1:
            ap_sum += precision_at_k(rel_ranked, i)
    return ap_sum / total_relevant

def f1_at_k(rel_ranked: List[int], k: int, total_relevant: int) -> float:
    """F1@K = 2 * P@K * R@K / (P@K + R@K)."""
    p = precision_at_k(rel_ranked, k)
    r = recall_at_k(rel_ranked, k, total_relevant)
    if p + r == 0:
        return 0.0
    return 2.0 * p * r / (p + r)

def mean_average_precision(all_ap: List[float]) -> float:
    """MAP = mean(AP) over queries."""
    return sum(all_ap) / len(all_ap) if all_ap else 0.0

def mean_reciprocal_rank(all_rel_ranked: List[List[int]]) -> float:
    """
    MRR = average over queries of 1/rank_of_first_relevant (rank is 1-based).
    If a query has no relevant in its ranking, its contribution is 0.
    """
    rr_sum = 0.0
    for rels in all_rel_ranked:
        rr = 0.0
        for i, r in enumerate(rels, start=1):
            if r == 1:
                rr = 1.0 / i
                break
        rr_sum += rr
    return rr_sum / len(all_rel_ranked) if all_rel_ranked else 0.0

def dcg_at_k(rel_ranked: List[int], k: int) -> float:
    """
    DCG@K with gains = 2^rel - 1 and log2 discount (positions 1-based):
      DCG@K = sum_{i=1..K} (2^{rel_i} - 1) / log2(i + 1)
    """
    import math
    if k <= 0:
        return 0.0
    dcg = 0.0
    upto = min(k, len(rel_ranked))
    for i in range(1, upto + 1):
        rel = rel_ranked[i - 1]
        gain = (2 ** rel) - 1
        dcg += gain / math.log2(i + 1)
    return dcg

def ndcg_at_k(rel_ranked: List[int], k: int) -> float:
    """NDCG@K = DCG@K / IDCG@K, where IDCG is DCG of rels sorted descending."""
    ideal = sorted(rel_ranked, reverse=True)
    dcg = dcg_at_k(rel_ranked, k)
    idcg = dcg_at_k(ideal, k)
    if idcg == 0.0:
        return 0.0
    return dcg / idcg

# Helpers to integrate with system 
def build_rel_vector(ranked_pids: List[str], gt_for_q: Dict[str, int]) -> Tuple[List[int], int]:
    """
    Convert a ranked list of pids into a relevance vector and compute total relevant.
    Inputs:
      - ranked_pids: list of doc ids (pids) in the returned order of your system.
      - gt_for_q: dict mapping pid -> 1/0 (ground truth relevance for ONE query).
    Returns:
      - rel_ranked: list[int], each element 1 if pid is relevant else 0 (aligned with ranked_pids)
      - total_relevant: total number of relevant docs for this query in the corpus (sum(gt_for_q))
    """
    rel_ranked = [int(gt_for_q.get(pid, 0)) for pid in ranked_pids]
    total_relevant = sum(1 for v in gt_for_q.values() if v == 1)
    return rel_ranked, total_relevant

def evaluate_query_at_k(ranked_pids: List[str], gt_rels: Dict[str, int], k: int = 20) -> Dict[str, float]:
    """
    Evaluate precision, recall, AP, F1, NDCG, MRR for a ranked list.
    gt_rels: dict {pid: 1 or 0}, where 1 means relevant.
    """
    rel_pids = {pid for pid, lbl in gt_rels.items() if lbl > 0}
    retrieved = ranked_pids[:k]
    retrieved_rels = [1 if pid in rel_pids else 0 for pid in retrieved]

    # count relevant retrieved (overlap)
    rel_retrieved = sum(retrieved_rels)
    total_relevant = len(rel_pids)

    precision = rel_retrieved / k if k > 0 else 0.0
    recall = rel_retrieved / total_relevant if total_relevant > 0 else 0.0

    # Average Precision (using 1 + log2 weighting)
    precisions = []
    num_rel_so_far = 0
    for i, rel in enumerate(retrieved_rels, start=1):
        if rel:
            num_rel_so_far += 1
            precisions.append(num_rel_so_far / i)
    ap = sum(precisions) / total_relevant if total_relevant > 0 else 0.0

    # F1-score
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) else 0.0

    # Reciprocal rank
    rr = 0.0
    for i, rel in enumerate(retrieved_rels, start=1):
        if rel:
            rr = 1.0 / i
            break

    # NDCG@K
    def dcg(rels):
        return sum((2 ** r - 1) / math.log2(i + 2) for i, r in enumerate(rels))
    dcg_val = dcg(retrieved_rels)
    ideal_rels = sorted(retrieved_rels, reverse=True)
    idcg_val = dcg(ideal_rels)
    ndcg = dcg_val / idcg_val if idcg_val > 0 else 0.0

    return {
        f"P@{k}": precision,
        f"R@{k}": recall,
        f"AP@{k}": ap,
        f"F1@{k}": f1,
        f"NDCG@{k}": ndcg,
        "MRR": rr,
        "total_relevant": total_relevant,
        "returned": len(retrieved)
    }

def evaluate_multiple_at_k(
    results_by_qid: Dict[str, List[str]],
    gt_all: Dict[str, Dict[str, int]],
    k: int = 20
) -> Dict[str, Any]:
    
    """Evaluate multiple queries at cut-off K."""
    per_query: Dict[str, Dict[str, float]] = {}
    ap_list: List[float] = []
    rel_lists: List[List[int]] = []
    ndcgs: List[float] = []

    for qid, ranked_pids in results_by_qid.items():
        gt_for_q = gt_all.get(qid, {})
        rel_ranked, total_rel = build_rel_vector(ranked_pids, gt_for_q)

        m = {
            f"P@{k}": precision_at_k(rel_ranked, k),
            f"R@{k}": recall_at_k(rel_ranked, k, total_rel),
            f"AP@{k}": average_precision_at_k(rel_ranked, k, total_rel),
            f"F1@{k}": f1_at_k(rel_ranked, k, total_rel),
            f"NDCG@{k}": ndcg_at_k(rel_ranked, k),
            "MRR": mean_reciprocal_rank([rel_ranked]),
            "total_relevant": total_rel,
            "returned": len(ranked_pids),
        }
        per_query[qid] = m
        ap_list.append(m[f"AP@{k}"])
        rel_lists.append(rel_ranked)
        ndcgs.append(m[f"NDCG@{k}"])

    summary = {
        "K": k,
        "MAP": mean_average_precision(ap_list),
        "MRR": mean_reciprocal_rank(rel_lists),
        f"mean_NDCG@{k}": (sum(ndcgs) / len(ndcgs) if ndcgs else 0.0),
    }
    return {"summary": summary, "per_query": per_query}

### **STEP 2 — Applying the Evaluation Metrics:**

In [6]:
import csv
from pathlib import Path

# Config
K = 20
VAL_PATH = DATA_DIR / "validation_labels.csv"

# Load ranked list (as appears in CSV order) and labels
ranked_by_qid = defaultdict(list)
gt_by_qid = defaultdict(dict)

if not VAL_PATH.exists():
    raise FileNotFoundError(f"Add validation_labels.csv to data/ directory for evaluation. File not found: {VAL_PATH}")

with VAL_PATH.open("r", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        qid = str(row["query_id"]).strip()
        pid = str(row["pid"]).strip()
        lab = int(row["labels"])  # 1/0
        ranked_by_qid[qid].append(pid)  
        gt_by_qid[qid][pid] = lab       


In [7]:
per_query_metrics = {}
ap_list = []
rel_lists_for_mrr = []
ndcgs = []

for qid, ranked_pids in ranked_by_qid.items():
    k_use = min(K, len(ranked_pids))
    gt_for_q = gt_by_qid[qid]

    # build the rel vector for the first K items as provided by the CSV order
    rel_ranked = [int(gt_for_q.get(pid, 0)) for pid in ranked_pids[:k_use]]

    m = evaluate_query_at_k(ranked_pids, gt_for_q, k=K)

    # store rounded
    per_query_metrics[qid] = {k: (round(v, 3) if isinstance(v, float) else v) for k, v in m.items()}

    ap_list.append(average_precision_at_k(rel_ranked, k_use, sum(gt_for_q.values())))
    rel_lists_for_mrr.append(rel_ranked)
    ndcgs.append(ndcg_at_k(rel_ranked, k_use))

summary = {
    "K": K,
    "MAP": mean_average_precision(ap_list),
    "MRR": mean_reciprocal_rank(rel_lists_for_mrr),
    f"mean_NDCG@{K}": (sum(ndcgs) / len(ndcgs) if ndcgs else 0.0),
}
summary_rounded = {k: (round(v, 3) if isinstance(v, float) else v) for k, v in summary.items()}


out_struct = {
    "query_1": per_query_metrics.get("1", {}),
    "query_2": per_query_metrics.get("2", {}),
    "summary": summary_rounded,
}

print(json.dumps(out_struct, indent=2))


{
  "query_1": {
    "P@20": 0.65,
    "R@20": 1.0,
    "AP@20": 0.694,
    "F1@20": 0.788,
    "NDCG@20": 0.873,
    "MRR": 1.0,
    "total_relevant": 13,
    "returned": 20
  },
  "query_2": {
    "P@20": 0.5,
    "R@20": 1.0,
    "AP@20": 0.627,
    "F1@20": 0.667,
    "NDCG@20": 0.833,
    "MRR": 1.0,
    "total_relevant": 10,
    "returned": 20
  },
  "summary": {
    "K": 20,
    "MAP": 0.66,
    "MRR": 1.0,
    "mean_NDCG@20": 0.853
  }
}


### **STEP 3 — Expert judges:**

#### **Part a)**

In [8]:
ANN_DIR = DATA_DIR / "annotations"
ANN_DIR.mkdir(parents=True, exist_ok=True)

PROPOSED_PATH = DATA_DIR / "index" / "proposed_test_queries.json"   # from Part 1.2
LABEL_TEMPLATE = ANN_DIR / "queries_label_template.csv"       

# Config
K = 10
OVERWRITE = False  

if not PROPOSED_PATH.exists():
    raise FileNotFoundError(f"Expected your proposed queries at: {PROPOSED_PATH}")

proposed = json.loads(PROPOSED_PATH.read_text(encoding="utf-8"))
queries = proposed.get("queries") or proposed.get("proposed") or proposed
if not isinstance(queries, list) or not queries:
    raise ValueError("No queries found in proposed_test_queries.json")

# If the file already exists, don't overwrite (to keep the whole team on the same 50 rows)
if LABEL_TEMPLATE.exists() and not OVERWRITE:
    print(f"Labeling template already exists, not overwritten:\n  {LABEL_TEMPLATE}")
    print("If you need to regenerate it, delete the file or set OVERWRITE = True.")
else:
    rows = []
    qid_counter = 1
    for q in queries:
        qid = str(qid_counter)
        hits = search_tfidf(q, k=K) 
        for rank, rec in enumerate(hits, start=1):
            rows.append({
                "query_id": qid,
                "query_text": q,
                "rank": rank,
                "pid": rec.get("pid"),
                "title": (rec.get("title") or "")[:120],
                "brand": rec.get("brand"),
                "category": rec.get("category"),
                "sub_category": rec.get("sub_category"),
                "url": rec.get("url"),
                "label": ""  # we will fill with 1 (relevant) or 0 (not relevant) manually
            })
        qid_counter += 1

    # Write CSV template
    with LABEL_TEMPLATE.open("w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=list(rows[0].keys()))
        writer.writeheader()
        writer.writerows(rows)

Labeling template already exists, not overwritten:
  C:\Users\Pol\Documents\POL\UNI\WEB\irwa-search-engine\data\annotations\queries_label_template.csv
If you need to regenerate it, delete the file or set OVERWRITE = True.


#### **Part b)**

In [9]:
import csv, json
from pathlib import Path
from collections import defaultdict

# Paths
LABELED_PATH = DATA_DIR / "annotations" / "queries_label_template.csv"  # already filled
OUT_JSON     = DATA_DIR / "index" / "eval_my_queries.json"
K = 10 

if not LABELED_PATH.exists():
    raise FileNotFoundError(f"Cannot find labeled file at: {LABELED_PATH}")

# Build structures
ranked_by_qid = defaultdict(list)
gt_by_qid = defaultdict(dict)
qtext_by_qid = {}

with LABELED_PATH.open("r", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        qid   = str(row["query_id"]).strip()
        pid   = str(row["pid"]).strip()
        label = str(row.get("label", "")).strip()

        # store query text (keeps the last seen; all rows per qid share it)
        qtext_by_qid[qid] = row.get("query_text", "").strip()

        # preserve CSV order as ranking
        ranked_by_qid[qid].append(pid)

        # label must be int 0/1; if empty we will treat as 0
        try:
            lab = int(label)
        except:
            lab = 0
        gt_by_qid[qid][pid] = 1 if lab == 1 else 0

print(f"Loaded labeled queries from: {LABELED_PATH}")
print(f"Found {len(ranked_by_qid)} queries. Example qids: {sorted(ranked_by_qid.keys())[:5]}")


Loaded labeled queries from: C:\Users\Pol\Documents\POL\UNI\WEB\irwa-search-engine\data\annotations\queries_label_template.csv
Found 5 queries. Example qids: ['1', '2', '3', '4', '5']


In [10]:
import json
import pprint
from statistics import mean

per_query = {}
ap_list = []
rel_lists = []
ndcgs = []

for qid in sorted(ranked_by_qid.keys(), key=lambda x: int(x) if x.isdigit() else x):
    ranked_pids = ranked_by_qid[qid]
    gt_for_q    = gt_by_qid[qid]

    # Per-query full metrics at cutoff K
    m = evaluate_query_at_k(ranked_pids, gt_for_q, k=K)

    # Rounding
    per_query[qid] = {k: (round(v, 3) if isinstance(v, float) else v) for k, v in m.items()}

    # Build pieces for summary
    k_use = min(K, len(ranked_pids))
    rel_ranked = [int(gt_for_q.get(pid, 0)) for pid in ranked_pids[:k_use]]
    ap_list.append(average_precision_at_k(rel_ranked, k_use, sum(gt_for_q.values())))
    rel_lists.append(rel_ranked)
    ndcgs.append(ndcg_at_k(rel_ranked, k_use))

summary = {
    "K": K,
    "MAP": mean_average_precision(ap_list),
    "MRR": mean_reciprocal_rank(rel_lists),
    f"mean_NDCG@{K}": (sum(ndcgs) / len(ndcgs) if ndcgs else 0.0),
}
summary = {k: (round(v, 3) if isinstance(v, float) else v) for k, v in summary.items()}

# Save full-precision results
OUT_JSON.parent.mkdir(parents=True, exist_ok=True)
OUT_JSON.write_text(json.dumps({"per_query": per_query, "summary": summary}, ensure_ascii=False, indent=2), encoding="utf-8")

#print(json.dumps({"per_query": per_query, "summary": summary}, indent=2))

pp = pprint.PrettyPrinter(width=200, compact=False)
pp.pprint({"per_query": per_query, "summary": summary})

{'per_query': {'1': {'AP@10': 0.0, 'F1@10': 0.0, 'MRR': 0.0, 'NDCG@10': 0.0, 'P@10': 0.0, 'R@10': 0.0, 'returned': 10, 'total_relevant': 0},
               '2': {'AP@10': 0.0, 'F1@10': 0.0, 'MRR': 0.0, 'NDCG@10': 0.0, 'P@10': 0.0, 'R@10': 0.0, 'returned': 10, 'total_relevant': 0},
               '3': {'AP@10': 0.43, 'F1@10': 0.571, 'MRR': 0.25, 'NDCG@10': 0.588, 'P@10': 0.4, 'R@10': 1.0, 'returned': 10, 'total_relevant': 4},
               '4': {'AP@10': 0.97, 'F1@10': 0.889, 'MRR': 1.0, 'NDCG@10': 0.992, 'P@10': 0.8, 'R@10': 1.0, 'returned': 10, 'total_relevant': 8},
               '5': {'AP@10': 0.2, 'F1@10': 0.182, 'MRR': 0.2, 'NDCG@10': 0.387, 'P@10': 0.1, 'R@10': 1.0, 'returned': 10, 'total_relevant': 1}},
 'summary': {'K': 10, 'MAP': 0.32, 'MRR': 0.29, 'mean_NDCG@10': 0.393}}


#### **Part c)**

In [11]:
import math, csv, json
from pathlib import Path
from collections import defaultdict

K = 10
LABELED_CSV = DATA_DIR / "annotations" / "queries_label_template.csv"
OUT_JSON    = DATA_DIR / "index" / "ablation_results.json"
OUT_CSV     = DATA_DIR / "index" / "ablation_results.csv"

if not LABELED_CSV.exists():
    raise FileNotFoundError(f"Missing labeled file: {LABELED_CSV}")

# Loading labeled ranking info (for query texts and ground-truth labels)
qid_to_text = {}
gt_by_qid   = defaultdict(dict)        # qid -> {pid: 0/1}
pids_by_qid = defaultdict(list)    
with LABELED_CSV.open("r", encoding="utf-8") as f:
    rdr = csv.DictReader(f)
    for row in rdr:
        qid   = str(row["query_id"]).strip()
        pid   = str(row["pid"]).strip()
        qtext = (row.get("query_text") or "").strip()
        lab_s = str(row.get("label") or "0").strip()
        lab   = 1 if lab_s == "1" else 0
        qid_to_text[qid] = qtext
        gt_by_qid[qid][pid] = lab
        pids_by_qid[qid].append(pid)

# Helpers to build TF-IDF ranker with field weights
from collections import defaultdict as dd

def _tokens_from_fields(record, fields):
    per_field_counts = {}
    for f in fields:
        txt = record.get(f)
        if not txt:
            continue
        counts = dd(int)
        for t in str(txt).split():
            counts[t] += 1
        if counts:
            per_field_counts[f] = dict(counts)
    return per_field_counts

def _tf_log2(freq):
    return 0.0 if freq <= 0 else 1.0 + math.log(freq, 2)

def _idf_log2(df_i, N):
    return 0.0 if df_i <= 0 or N <= 0 else math.log(N/df_i, 2)

def build_tfidf_ranker(field_weights, fields=("title_clean","description_clean","metadata_clean")):
    """
    Builds a cosine-normalized TF-IDF ranker with per-field weights.
    Returns: rank(query, k=K, restrict_doc_ids=None) -> [(doc_id, score), ...]
    """
    N = len(docs)
    tf_post    = dd(dict)   # term -> {doc_id: weighted_raw_tf}
    df_counts  = dd(int)
    doc_norms  = [0.0]*N

    # aggregate weighted term counts per doc
    for d_id, rec in enumerate(docs):
        per_field = _tokens_from_fields(rec, fields)
        agg = dd(float)
        for fname, counts in per_field.items():
            w = field_weights.get(fname, 1.0)
            for term, c in counts.items():
                agg[term] += w * c
        for term, f in agg.items():
            tf_post[term][d_id] = f

    # df
    for term, posting in tf_post.items():
        df_counts[term] = len(posting)

    # doc norms
    for term, posting in tf_post.items():
        idf = _idf_log2(df_counts[term], N)
        if idf == 0: 
            continue
        for d_id, f in posting.items():
            wdt = _tf_log2(f) * idf
            if wdt != 0:
                doc_norms[d_id] += wdt*wdt
    doc_norms = [math.sqrt(x) if x>0 else 0.0 for x in doc_norms]

    def rank(query, k=K, restrict_doc_ids=None):
        proc = preprocess_text_field(query or "")
        q_terms = proc["tokens"]
        if not q_terms:
            return []

        # query vector
        q_tf = dd(int)
        for t in q_terms: q_tf[t]+=1
        q_w = {}
        q_norm_sq = 0.0
        for t, fq in q_tf.items():
            df = df_counts.get(t, 0)
            if df <= 0:
                continue
            wt = _tf_log2(fq) * _idf_log2(df, N)
            if wt != 0:
                q_w[t] = wt
                q_norm_sq += wt*wt
        q_norm = math.sqrt(q_norm_sq) if q_norm_sq>0 else 0.0
        if q_norm == 0.0:
            return []

        scores = dd(float)
        for t, wq in q_w.items():
            posting = tf_post.get(t)
            if not posting:
                continue
            idf = _idf_log2(df_counts[t], N)
            if idf == 0:
                continue
            for d_id, f in posting.items():
                if restrict_doc_ids is not None and d_id not in restrict_doc_ids:
                    continue
                wdt = _tf_log2(f) * idf
                if wdt != 0:
                    scores[d_id] += wq * wdt

        ranked = []
        for d_id, dot in scores.items():
            denom = doc_norms[d_id]*q_norm
            if denom>0:
                ranked.append((d_id, dot/denom))
        ranked.sort(key=lambda x: x[1], reverse=True)
        return ranked[:k]
    return rank

# Configs to test
CONFIGS = [
    {"name":"tfidf",         "field_weights":{"title_clean":2.0, "description_clean":1.0, "metadata_clean":0.7}, "and_filter":False},
    {"name":"tfidf_and",     "field_weights":{"title_clean":2.0, "description_clean":1.0, "metadata_clean":0.7}, "and_filter":True},
    {"name":"tfidf_title3",  "field_weights":{"title_clean":3.0, "description_clean":1.0, "metadata_clean":0.7}, "and_filter":False},
    {"name":"tfidf_no_meta", "field_weights":{"title_clean":2.0, "description_clean":1.0, "metadata_clean":0.0}, "and_filter":False},
]

# Run ablations
from statistics import mean

results_table = []
per_query_detailed = {}  # config -> {qid -> metrics dict}

for cfg in CONFIGS:
    ranker = build_tfidf_ranker(cfg["field_weights"])
    per_q_metrics = {}
    map_list, mrr_list, ndcg_list = [], [], []

    for qid in sorted(qid_to_text.keys(), key=lambda x: int(x) if x.isdigit() else x):
        qtext = qid_to_text[qid]

        restrict_ids = None
        if cfg["and_filter"]:
            # Boolean AND candidates
            and_hits = search_and(qtext, k=50_000)  # uses our boolean index
            cand_pids = {h["pid"] for h in and_hits if h.get("pid")}
            restrict_ids = {i for i, rec in enumerate(docs) if rec.get("pid") in cand_pids}

        ranked_pairs = ranker(qtext, k=K, restrict_doc_ids=restrict_ids)
        ranked_pids  = [docs[d_id].get("pid") for d_id, _ in ranked_pairs]
        m = evaluate_query_at_k(ranked_pids, gt_by_qid[qid], k=K)
        per_q_metrics[qid] = m

        map_list.append(m[f"AP@{K}"])
        mrr_list.append(m["MRR"])
        ndcg_list.append(m[f"NDCG@{K}"])

    per_query_detailed[cfg["name"]] = per_q_metrics
    results_table.append({
        "config": cfg["name"],
        "MAP": round(mean(map_list), 3) if map_list else 0.0,
        "MRR": round(mean(mrr_list), 3) if mrr_list else 0.0,
        f"mean_NDCG@{K}": round(mean(ndcg_list), 3) if ndcg_list else 0.0,
    })

# Save
OUT_JSON.parent.mkdir(parents=True, exist_ok=True)
OUT_JSON.write_text(json.dumps({"summary": results_table, "per_query": per_query_detailed}, ensure_ascii=False, indent=2), encoding="utf-8")

5021

In [12]:
#Display results (sorted by MAP) and save a CSV
import json, csv
from pathlib import Path

# Load the results (the "just-written")
ablation_path = DATA_DIR / "index" / "ablation_results.json"
res = json.loads(ablation_path.read_text(encoding="utf-8"))

summary = sorted(res["summary"], key=lambda x: x["MAP"], reverse=True)
print("Ablation summary (higher = better):")
print(json.dumps(summary, indent=2))

# Write a compact CSV for the report
csv_out = DATA_DIR / "index" / "ablation_results.csv"
with csv_out.open("w", newline="", encoding="utf-8") as f:
    w = csv.DictWriter(f, fieldnames=summary[0].keys())
    w.writeheader()
    w.writerows(summary)
print(f"\nSaved: {csv_out}")

# import pprint
# pp = pprint.PrettyPrinter(width=160)
# pp.pprint(res["per_query"])

Ablation summary (higher = better):
[
  {
    "config": "tfidf_and",
    "MAP": 0.6,
    "MRR": 0.6,
    "mean_NDCG@10": 0.6
  },
  {
    "config": "tfidf_no_meta",
    "MAP": 0.483,
    "MRR": 0.6,
    "mean_NDCG@10": 0.565
  },
  {
    "config": "tfidf_title3",
    "MAP": 0.326,
    "MRR": 0.29,
    "mean_NDCG@10": 0.395
  },
  {
    "config": "tfidf",
    "MAP": 0.32,
    "MRR": 0.29,
    "mean_NDCG@10": 0.393
  }
]

Saved: C:\Users\Pol\Documents\POL\UNI\WEB\irwa-search-engine\data\index\ablation_results.csv
